<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
# 📊 מדריך PySpark מקיף - Pivot, PartitionBy ו-MapType

## 📚 סקירה כללית

מחברת זו מכסה שלושה נושאים חשובים ב-PySpark:

1. **🔄 Pivot & Unpivot** - סיבוב נתונים מעמודות לשורות ולהיפך - [Pivot & Unpivot](https://sparkbyexamples.com/pyspark/pyspark-pivot-and-unpivot-dataframe/)
2. **💾 PartitionBy** - חלוקת נתונים לקבצים מרובים על הדיסק - [PartitionBy](https://sparkbyexamples.com/pyspark/pyspark-partitionby-example/)
3. **🗺️ MapType** - עבודה עם מבני נתונים מסוג Dictionary/Map - [MapType](https://sparkbyexamples.com/pyspark/pyspark-maptype-dict-examples/)

---

## 🔄 חלק 1: Pivot ו-Unpivot ב-DataFrame

### מהו Pivot?

הפונקציה `pivot()` משמשת לסיבוב/טרנספוזיציה של נתונים מעמודה אחת למספר עמודות DataFrame. Pivot הוא אגרגציה שבה אחד מערכי עמודות הקיבוץ מועבר לעמודות בודדות עם נתונים מובחנים.

### תחביר (Syntax)

```python
pivot_df = original_df.groupBy("grouping_column").pivot("pivot_column").agg({"agg_column": "agg_function"})
```

- **grouping_column**: העמודה המשמשת לקיבוץ
- **pivot_column**: העמודה שערכיה המובחנים יהפכו לעמודות חדשות
- **agg_column**: העמודה שעליה מבוצעת האגרגציה (לדוגמה, שימוש בפונקציות כמו `sum`, `avg` וכו')

### ייבוא ספריות והגדרת Spark Session

In [ ]:
# ייבוא ספריות
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

# יצירת Spark Session
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

### יצירת DataFrame לדוגמה

In [ ]:
# יצירת נתונים לדוגמה
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), 
        ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), 
        ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), 
        ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]

columns= ["Product","Amount","Country"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)

### 🔄 ביצוע Pivot על DataFrame

PySpark SQL מספק פונקציית `pivot()` לסיבוב הנתונים מעמודה אחת למספר עמודות. זוהי אגרגציה שבה אחד מערכי עמודות הקיבוץ מועבר לעמודות בודדות עם נתונים מובחנים.

כדי לקבל את הסכום הכולל המיוצא לכל מדינה עבור כל מוצר, נבצע קיבוץ לפי `Product`, pivot לפי `Country`, וסכום של `Amount`.

In [ ]:
# ביצוע pivot
pivotDF = df.groupBy("Product").pivot("Country").sum("Amount")
pivotDF.printSchema()
pivotDF.show(truncate=False)

הפעולה תעביר את המדינות משורות DataFrame לעמודות ותייצר פלט שבו כל נתון שאינו קיים מיוצג כ-`null` כברירת מחדל.

### ⚡ שיפור ביצועים ב-PySpark 2.0

גרסה 2.0 ומעלה שיפרה את הביצועים של Pivot. עם זאת, אם אתה משתמש בגרסה נמוכה יותר, שים לב ש-pivot היא פעולה יקרה מאוד, לכן מומלץ לספק נתוני עמודות (אם ידועים) כארגומנט לפונקציה כפי שמוצג להלן.

#### גישה 1: שימוש בפונקציה עם נתוני עמודות

In [ ]:
# Pivot עם ציון מדינות מראש
countries = ["USA","China","Canada","Mexico"]
pivotDF = df.groupBy("Product").pivot("Country", countries).sum("Amount")
pivotDF.show(truncate=False)

#### גישה 2: אגרגציה דו-שלבית

גישה נוספת היא לבצע אגרגציה דו-שלבית. PySpark 2.0 משתמש ביישום זה כדי לשפר את הביצועים (Spark-13749).

In [ ]:
# אגרגציה דו-שלבית
pivotDF = df.groupBy("Product","Country") \
      .sum("Amount") \
      .groupBy("Product") \
      .pivot("Country") \
      .sum("sum(Amount)")
pivotDF.show(truncate=False)

שתי הדוגמאות לעיל מחזירות את אותו פלט אך עם ביצועים טובים יותר.

### 🔙 Unpivot של PySpark DataFrame

Unpivot הוא פעולה הפוכה. ניתן להשיג זאת על ידי סיבוב ערכי עמודות לערכי שורות. PySpark SQL אינו מכיל פונקציית unpivot, לכן נשתמש בפונקציה `stack()` במקום.

הקוד הבא ממיר עמודת pivot "country" לשורות.

In [ ]:
# Unpivot - המרת עמודות למדינות בחזרה לשורות
from pyspark.sql.functions import expr

unpivotExpr = "stack(3, 'Canada', Canada, 'China', China, 'Mexico', Mexico) as (Country,Total)"
unPivotDF = pivotDF.select("Product", expr(unpivotExpr)) \
    .where("Total is not null")
unPivotDF.show(truncate=False)

הפעולה ממירה עמודת pivot "country" לשורות.

### ❓ שאלות נפוצות על pivot() ו-unpivot()

#### האם ניתן לבצע transpose או pivot של PySpark DataFrame ללא אגרגציה?

כמובן שכן, אך למרבה הצער, אי אפשר להשיג זאת באמצעות פונקציית Pivot. עם זאת, pivoting או transposing מבנה DataFrame ללא אגרגציה משורות לעמודות ומעמודות לשורות ניתן לביצוע בקלות באמצעות PySpark ו-Scala hack. אנא עיין בדוגמה ב-stackoverflow.

#### מה קורה אם יש ערכים כפולים עבור עמודת ה-pivot?

פעולת `pivot` דורשת שילובים ייחודיים של עמודות קיבוץ ו-pivot. אם יש ערכים כפולים, ייתכן שתצטרך לבצע אגרגציה (למשל, שימוש בפרמטר `agg`) כדי לפתור את הקונפליקט.

#### כיצד פעולת pivot מטפלת בערכים חסרים?

פעולת `pivot` ממלאת ערכים חסרים עם null. אם אתה צריך לטפל בערכים חסרים באופן שונה, אתה יכול להשתמש בשיטות כמו `fillna` או `na.drop` על ה-DataFrame ה-pivoted.

### 📝 סיכום - Pivot & Unpivot

ראינו כיצד ל-Pivot DataFrame עם PySpark לדוגמה ול-Unpivot בחזרה באמצעות פונקציות SQL. וראינו גם כיצד שינויי PySpark 2.0 שיפרו את הביצועים על ידי ביצוע אגרגציה דו-שלבית.

---

## 💾 חלק 2: PySpark partitionBy() - כתיבה לדיסק

### מהו Partition ב-PySpark?

PySpark `partitionBy()` היא פונקציה של מחלקה `pyspark.sql.DataFrameWriter` המשמשת לחלוקת הנתונים הגדולים (DataFrame) לקבצים קטנים יותר על בסיס עמודה אחת או יותר תוך כתיבה לדיסק. בואו נראה כיצד להשתמש בזה עם דוגמאות Python.

### יתרונות Partition

חלוקת הנתונים על מערכת הקבצים היא דרך לשפר את הביצועים של השאילתות כאשר מתמודדים עם מערך נתונים גדול, למשל Data lake.

כפי שאתה מודע, PySpark מתוכנן לעבד מערכי נתונים גדולים מהר פי 100 מהעיבוד המסורתי. זה לא היה אפשרי ללא partition. להלן מספר יתרונות של שימוש ב-partitions של PySpark על זיכרון או על דיסק:

- **גישה מהירה לנתונים** ⚡
- **יכולת לבצע פעולות על מערך נתונים קטן יותר** 📊

Partition במנוחה (דיסק) היא תכונה של מסדי נתונים רבים ומסגרות עיבוד נתונים, והיא מפתח להפוך עבודות לעבוד בקנה מידה.

### 🔀 PySpark תומך ב-Partition בשתי דרכים

1. **Partition in memory (DataFrame)**: אתה יכול לחלק או לבצע repartition של DataFrame על ידי קריאה ל-`repartition()` או `coalesce()`
2. **Partition on disk (File system)**: בזמן כתיבת PySpark DataFrame בחזרה לדיסק, אתה יכול לבחור כיצד לחלק את הנתונים על בסיס עמודות באמצעות `partitionBy()` של `pyspark.sql.DataFrameWriter`. זה דומה ל-Hives partitions scheme.

### 📂 יצירת DataFrame לדוגמה

בואו ניצור DataFrame על ידי קריאת קובץ CSV. עקוב אחר Github zipcodes.csv כדי להוריד את קובץ ה-CSV.

In [ ]:
# יצירת DataFrame מקובץ CSV
df = spark.read.option("header",True) \
        .csv("../data/simple-zipcodes.csv")
df.printSchema()

מה-DataFrame לעיל, אשתמש ב-`state` כמפתח partition לדוגמאות שלנו להלן.

### 💾 PySpark partitionBy()

ב-PySpark, טרנספורמציית `partitionBy()` משמשת לחלוקת נתונים ב-RDD או DataFrame על בסיס המחיצה שצוינה. היא מיושמת בדרך כלל לאחר פעולות מסוימות כמו `groupBy()` או `join()` כדי לשלוט בהפצה של הנתונים על פני מחיצות.

ה-`partitionBy()` זמין במחלקה `DataFrameWriter`, ולכן משתמשים בה כדי לכתוב את נתוני ה-partition לדיסק.

#### תחביר (Syntax)

```python
partitionBy(self, *cols)
```

כאשר אתה כותב PySpark DataFrame לדיסק על ידי קריאה ל-`partitionBy()`, PySpark מפצל את הרשומות על בסיס עמודת ה-partition ושומר כל נתוני partition בתת-ספרייה.

#### דוגמה: Partition עם עמודה אחת

In [ ]:
# דוגמה ל-partitionBy()
df.write.option("header",True) \
        .partitionBy("state") \
        .mode("overwrite") \
        .csv("tmp/zipcodes-state")

הנתונים שלי כוללים שש מדינות, לכן נוצרו שש ספריות על הדיסק. שם תת-הספרייה יהיה עמודת ה-partition והערך שלה (partition column=value).

ניתן לאמת זאת על ידי הצגת הקבצים בספרייה:

In [ ]:
# הצגת מבנה הקבצים (בשורת הפקודה)
# !ls -lrt tmp/zipcodes-state

**הערה חשובה:** בזמן כתיבת הנתונים כ-partitions, PySpark מבטל את עמודת ה-partition בקובץ הנתונים ומוסיף את עמודת ה-partition והערך לשם התיקייה, וחוסך מקום אחסון.

בכל ספרייה, ייתכנו חלק קובץ אחד או יותר (שכן מערך הנתונים שלנו קטן, כל הרשומות לכל `state` נשמרות בחלק קובץ אחד). ניתן לשנות התנהגות זו על ידי ציון מספר ה-partitions (חלקי קבצים) שתרצה לכל `state` כארגומנט לשיטת `repartition()`.

### 📚 PySpark partitionBy() - עמודות מרובות

אתה יכול גם ליצור partitions על עמודות מרובות באמצעות PySpark `partitionBy()`. פשוט העבר את העמודות שברצונך לחלק כארגומנטים לשיטה זו.

In [ ]:
# Partition עם עמודות מרובות
df.write.option("header",True) \
        .partitionBy("state","city") \
        .mode("overwrite") \
        .csv("tmp/zipcodes-state-city")

זה יוצר היררכיית תיקיות לכל partition; ציינו את ה-partition הראשון כ-`state` ואחריו `city`, לכן היא יוצרת תיקיית `city` בתוך תיקיית `state` (תיקייה אחת עבור כל `city` ב-`state`).

ניתן לאמת זאת על ידי הצגת המבנה:

In [ ]:
# הצגת מבנה היררכי של הקבצים
!ls -lrt tmp/zipcodes-state-city/state=AL

### 🔄 שימוש ב-repartition() יחד עם partitionBy()

`repartition()` משמש להגדלה או הקטנה של מספר ה-partitions בזיכרון. כאשר משתמשים ב-`repartition()` עם `partitionBy()`, הוא מפצל עוד יותר למספר partitions מרובים על בסיס נתוני עמודות.

In [ ]:
# שימוש ב-repartition() ו-partitionBy() יחד
df.repartition(2) \
        .write.option("header",True) \
        .partitionBy("state") \
        .mode("overwrite") \
        .csv("tmp/zipcodes-state-more")

לאחר בדיקת התיקייה שבה הנתונים נכתבים, תבחין שיש רק שני קבצי partition עבור כל מדינה. לאחר שמערך הנתונים כולל שש מדינות ייחודיות ויש שני partitions בזיכרון המוקצים לכל מדינה, הקוד יוצר מקסימום של 12 קבצי partition.

### 🎯 Data Skew - שליטה במספר הרשומות לכל Partition

השתמש באפשרות `maxRecordsPerFile` אם תרצה לשלוט במספר הרשומות עבור כל partition. זה שימושי במיוחד כאשר הנתונים שלך skewed (כלומר, יש partitions עם מספר רשומות נמוך מאוד ו-partitions אחרים עם מספר רשומות גבוה).

In [ ]:
# שליטה במספר רשומות לכל קובץ
df.write.option("header",True) \
        .option("maxRecordsPerFile", 2) \
        .partitionBy("state") \
        .mode("overwrite") \
        .csv("tmp/zipcodes-state")

הדוגמה לעיל יוצרת קבצי part מרובים עבור כל `state`, וכל קובץ part מכיל רק 2 רשומות.

### 📖 קריאת Partition ספציפי

קריאות מהירות יותר בהרבה על נתונים ש-partitioned. קטע הקוד הזה מאחזר את הנתונים מ-partition ספציפי `state=AL` ו-`city=SPRINGVILLE`. כאן, הוא פשוט קורא את הנתונים מאותה תיקייה ספציפית במקום לסרוק קובץ שלם (כשלא partitioned).

In [ ]:
# קריאה מנתוני partition ספציפיים
dfSinglePart = spark.read.option("header",True) \
            .csv("tmp/zipcodes-state-city/state=AZ/city=MESA")
dfSinglePart.printSchema()
dfSinglePart.show()

בעת קריאת נתוני Partition ספציפיים ל-DataFrame, הוא לא שומר את עמודות ה-partition על DataFrame, ולכן ב-DataFrame חסרות עמודות `state` ו-`city`.

### 🔍 PySpark SQL - קריאת נתוני Partition

זו דוגמה לכתיבת Spark DataFrame על ידי שמירת עמודות ה-partition על DataFrame.

In [ ]:
# קריאה מנתוני partition תוך שימוש ב-SQL
parqDF = spark.read.option("header",True).csv("tmp/zipcodes-state")
parqDF.createOrReplaceTempView("ZIPCODE")
spark.sql("select * from ZIPCODE  where state='AL' and city = 'SPRINGVILLE'") \
    .show()


ביצוע השאילתה הזה גם מהיר משמעותית מהשאילתה ללא partition. הוא מסנן את הנתונים תחילה ב-`state`, ואז מיישם מסננים לעמודה `city` מבלי לסרוק את מערך הנתונים השלם.

### 🤔 כיצד לבחור עמודת Partition בעת כתיבה למערכת הקבצים?

בעת יצירת partitions עליך להיות זהיר מאוד עם מספר ה-partitions שתיצור, שכן יצירת יותר מדי partitions יוצרת יותר מדי תת-ספריות ב-HDFS, מה שמביא לעומס ומינהל מיותרים ל-NameNode (אם אתה משתמש ב-Hadoop), שכן הוא חייב לשמור את כל המטא-דאטה של מערכת הקבצים בזיכרון.

### דוגמה למצב בעייתי:

קחו בחשבון תרחיש שבו יש לכם טבלת אוכלוסין אמריקאית עם מיקוד, ערים, מדינות ועמודות נוספות. שימוש ב-state כמפתח partition מחלק את הטבלה לכ-50 partitions. כתוצאה מכך, כשמחפשים מיקוד בתוך מדינה ספציפית (למשל, state='CA' ו-zipCode='92704'), התהליך מואץ מכיוון שהוא כולל רק סריקה בספרייה המקושרת ל-partition state='CA'.

### הגדרה הבעייתית:

**Partition על zipcode אינו אפשרות טובה מכיוון שייתכן שתסיים עם יותר מדי partitions.**

### דוגמה טובה:

**דוגמה טובה אחרת של partition היא בעמודת התאריך. באופן אידיאלי, עליך לבצע partition לפי שנה/חודש אך לא לפי תאריך.**

### ❓ שאלות נפוצות על partitionBy()

#### 1. מה ההבדל בין partitionBy() ל-groupBy() ב-PySpark?

**`partitionBy()`** משמש לארגון פיזי של הנתונים על הדיסק בעת כתיבה למערכת קבצים, בעוד **`groupBy()`** משמש לקיבוץ לוגי של נתונים בתוך DataFrame.

#### 2. האם ניתן להשתמש בעמודות מרובות עם partitionBy()?

כן, ניתן לציין עמודות מרובות בפונקציה `partitionBy()` כדי ליצור מבנה תיקיות היררכי. לדוגמה:

```python
df.write.partitionBy("column1", "column2").parquet("/path/to/output")
```

#### 3. כיצד partitioning משפיע על ביצועי שאילתות?

Partitioning יכול לשפר משמעותית את ביצועי השאילתה, במיוחד כאשר שואלים תתי קבוצות ספציפיות של נתונים. זה עוזר לדלג על נתונים לא רלוונטיים בעת קריאה, ומקטין את כמות הנתונים שצריך לעבד.

### 📝 סיכום - PartitionBy

בזמן שאתה יוצר Data Lake מ-Azure, HDFS או AWS, אתה צריך להבין כיצד לחלק את הנתונים שלך במנוחה (מערכת קבצים/דיסק). PySpark `partitionBy()` ו-`repartition()` עוזרים לך לחלק את הנתונים ולבטל את ה-Data Skew על מערכי הנתונים הגדולים שלך.

אני מקווה שזה נותן לך רעיון טוב יותר על partitions ב-PySpark.

---

## 🗺️ חלק 3: PySpark MapType (Dict) - שימוש ודוגמאות

### מהו PySpark MapType?

PySpark `MapType` (נקרא גם סוג map) הוא סוג נתונים לייצוג Python Dictionary (`dict`) לאחסון זוג key-value. אובייקט MapType כולל שלושה שדות:

- `keyType` - DataType
- `valueType` - DataType  
- `valueContainsNull` - Boolean (ברירת מחדל True)

### הרחבת מחלקת DataType

PySpark MapType משמש לייצוג זוג key-value דומה ל-Python Dictionary (Dict). הוא מרחיב את מחלקת DataType שהיא superclass של כל הסוגים ב-PySpark ולוקח שני ארגומנטים חובה:

- `keyType` - סוג המפתח
- `valueType` - סוג הערך

וארגומנט בוליאני אופציונלי אחד:

- `valueContainsNull` (ברירת מחדל True)

`keyType` ו-`valueType` יכולים להיות מכל סוג שמרחיב את DataType. לדוגמה: `StringType`, `IntegerType`, `ArrayType`, `MapType`, `StructType` (struct) וכו'.

### ⚙️ יצירת PySpark MapType

כדי להשתמש בסוג נתונים MapType תחילה, עליך לייבא אותו מ-`pyspark.sql.types.MapType` ולהשתמש בבנאי `MapType()` ליצירת אובייקט map.

#### נקודות מפתח:

- הפרמטר הראשון `keyType` משמש לציון סוג המפתח במפה
- הפרמטר השני `valueType` משמש לציון סוג הערך במפה
- הפרמטר השלישי `valueContainsNull` הוא סוג בוליאני אופציונלי המשמש לציון אם הערך של הפרמטר השני יכול לקבל ערכי `Null/None`
- המפתח של המפה לא יקבל ערכי `None/Null`
- PySpark מספק מספר פונקציות SQL לעבודה עם `MapType`

### 📝 דוגמה: יצירת MapType

In [ ]:
# ייבוא הספריות הנדרשות
from pyspark.sql.types import StringType, MapType

# יצירת MapType
mapCol = MapType(StringType(), StringType(), False)

### 🏗️ יצירת MapType מ-StructType

בואו נראה כיצד ליצור MapType באמצעות PySpark StructType ו-StructField. `StructType()` constructor לוקח list של StructField. StructField לוקח fieldname וסוג הערך.

In [ ]:
# יצירת MapType מ-StructType
from pyspark.sql.types import StructField, StructType, StringType, MapType

schema = StructType([
    StructField('name', StringType(), True),
    StructField('properties', MapType(StringType(), StringType()), True)
])

עכשיו בואו ניצור DataFrame באמצעות StructType schema לעיל.

In [ ]:
# יצירת DataFrame עם MapType
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

dataDictionary = [
        ('James', {'hair':'black', 'eye':'brown'}),
        ('Michael', {'hair':'brown', 'eye':None}),
        ('Robert', {'hair':'red', 'eye':'black'}),
        ('Washington', {'hair':'grey', 'eye':'grey'}),
        ('Jefferson', {'hair':'brown', 'eye':''})  
]

df = spark.createDataFrame(data=dataDictionary, schema=schema)
df.printSchema()
df.show(truncate=False)

### 🔍 גישה לאלמנטים של PySpark MapType

בואו נראה כיצד לחלץ את המפתח והערכים מעמודת Dictionary ב-PySpark DataFrame. כאן השתמשתי בטרנספורמציה של PySpark map כדי לקרוא את הערכים של `properties` (עמודת MapType).

In [ ]:
# גישה לאלמנטים של Map
df3 = df.rdd.map(lambda x: \
    (x.name, x.properties["hair"], x.properties["eye"])) \
    .toDF(["name", "hair", "eye"])
df3.printSchema()
df3.show()

בואו נשתמש בדרך נוספת לקבל את הערך של מפתח מ-Map באמצעות `getItem()` של עמודה Column. שיטה זו לוקחת מפתח כארגומנט ומחזירה ערך.

In [ ]:
# שימוש ב-getItem() לגישה לערכים
df.withColumn("hair", df.properties.getItem("hair")) \
  .withColumn("eye", df.properties.getItem("eye")) \
  .drop("properties") \
  .show()

df.withColumn("hair", df.properties["hair"]) \
  .withColumn("eye", df.properties["eye"]) \
  .drop("properties") \
  .show()

### 🔧 פונקציות MapType

להלן כמה מפונקציות MapType עם דוגמאות.

#### 🎯 explode() - הרחבת Map לשורות

בואו נחיל את פונקציה `explode()` על עמודת map ב-DataFrame כדי להרחיב כל זוג key-value לשורות נפרדות.

In [ ]:
# שימוש ב-explode להרחבת Map
from pyspark.sql.functions import explode

df.select(df.name, explode(df.properties)).show()

#### 🔑 map_keys() - קבלת כל מפתחות ה-Map

פונקציית `map_keys()` מחזירה את כל המפתחות מעמודת MapType כרשימה.

In [ ]:
# קבלת כל מפתחות ה-Map
from pyspark.sql.functions import map_keys

df.select(df.name, map_keys(df.properties)).show()

**אזהרה:** זה פועל לאט מאוד.

במקרה שתרצה לקבל את כל מפתחות ה-map כרשימת Python:

In [ ]:
# קבלת מפתחות כרשימת Python
from pyspark.sql.functions import explode, map_keys

keysDF = df.select(explode(map_keys(df.properties))).distinct()
keysList = keysDF.rdd.map(lambda x: x[0]).collect()
print(keysList)
# ['eye', 'hair']

#### 💎 map_values() - קבלת כל ערכי ה-Map

In [ ]:
# קבלת כל ערכי ה-Map
from pyspark.sql.functions import map_values

df.select(df.name, map_values(df.properties)).show()

### 📝 סיכום - MapType

MapType הוא מבנה נתונים של map המשמש לאחסון זוג key-value דומה ל-Python Dictionary (Dict). מפתחות וערכים של סוג map צריכים להיות מסוג שמרחיב DataType. מפתח של ה-map לא יקבל ערכי `null/None`, בעוד שמפה של המפתח יכולה לקבל ערכי `None/Null`.

---

## 🎓 סיכום כללי

במדריך זה למדנו שלושה נושאים חשובים ב-PySpark:

### 1. 🔄 Pivot & Unpivot
- סיבוב נתונים מעמודות לשורות ולהיפך
- שיפור ביצועים עם אגרגציה דו-שלבית
- שימוש ב-`stack()` ל-unpivot

### 2. 💾 PartitionBy
- חלוקת נתונים גדולים לקבצים קטנים יותר
- שיפור ביצועי שאילתות
- בחירה נכונה של עמודות partition
- טיפול ב-data skew

### 3. 🗺️ MapType
- ייצוג מבני Dictionary ב-PySpark
- גישה לערכים וmפתחות
- פונקציות עזר כמו `explode()`, `map_keys()`, `map_values()`

---

## 📚 משאבים נוספים

- [PySpark Documentation](https://spark.apache.org/docs/latest/api/python/)
- [SparkByExamples.com](https://sparkbyexamples.com)
- [Apache Spark GitHub](https://github.com/apache/spark)

---

### 💡 טיפים לעבודה עם PySpark

1. **תכנן מראש** - חשוב על מבנה הנתונים והשאילתות לפני יצירת partitions
2. **מדוד ביצועים** - השתמש ב-`explain()` ו-Spark UI להבנת ביצועי השאילתות
3. **נהל משאבים** - שים לב למספר ה-partitions והגודל שלהם
4. **בדוק את הנתונים** - השתמש ב-`show()`, `printSchema()` ו-`describe()` לבדיקת הנתונים

---

**Happy Learning! 🎉**